# 1-Data preprocessing

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import r2_score
import joblib


In [2]:
df = pd.read_csv("./data/properties.csv")

In [3]:
df.head()

,id,price,property_type,subproperty_type,region,province,locality,zip_code,latitude,longitude,...,fl_garden,garden_sqm,fl_swimming_pool,fl_floodzone,state_building,primary_energy_consumption_sqm,epc,heating_type,fl_double_glazing,cadastral_income
0,34221000,225000,APARTMENT,APARTMENT,Flanders,Antwerp,Antwerp,2050,51.217172,4.379982,...,0,0.0,0,0,MISSING,231.0,C,GAS,1,922.0
1,2104000,449000,HOUSE,HOUSE,Flanders,East Flanders,Gent,9185,51.174944,3.845248,...,0,0.0,0,0,MISSING,221.0,C,MISSING,1,406.0
2,34036000,335000,APARTMENT,APARTMENT,Brussels-Capital,Brussels,Brussels,1070,50.842043,4.334543,...,0,0.0,0,1,AS_NEW,NaN,MISSING,GAS,0,NaN
3,58496000,501000,HOUSE,HOUSE,Flanders,Antwerp,Turnhout,2275,51.238312,4.817192,...,0,0.0,0,1,MISSING,99.0,A,MISSING,0,NaN
4,48727000,982700,APARTMENT,DUPLEX,Wallonia,Walloon Brabant,Nivelles,1410,NaN,NaN,...,1,142.0,0,0,AS_NEW,19.0,A+,GAS,0,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75511 entries, 0 to 75510
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              75511 non-null  int64  
 1   price                           75511 non-null  int64  
 2   property_type                   75511 non-null  object 
 3   subproperty_type                75511 non-null  object 
 4   region                          75511 non-null  object 
 5   province                        75511 non-null  object 
 6   locality                        75511 non-null  object 
 7   zip_code                        75511 non-null  int64  
 8   latitude                        61413 non-null  float64
 9   longitude                       61413 non-null  float64
 10  construction_year               42120 non-null  float64
 11  total_area_sqm                  67896 non-null  float64
 12  surface_land_sqm                

In [5]:
# Counting null values for each variable
null_counts = df.isna().sum()
null_percentages = null_counts/ len(df) * 100
print (null_percentages)

id                                 0.000000
price                              0.000000
property_type                      0.000000
subproperty_type                   0.000000
region                             0.000000
province                           0.000000
locality                           0.000000
zip_code                           0.000000
latitude                          18.670128
longitude                         18.670128
construction_year                 44.220047
total_area_sqm                    10.084623
surface_land_sqm                  48.014197
nbr_frontages                     34.890281
nbr_bedrooms                       0.000000
equipped_kitchen                   0.000000
fl_furnished                       0.000000
fl_open_fire                       0.000000
fl_terrace                         0.000000
terrace_sqm                       17.401438
fl_garden                          0.000000
garden_sqm                         3.892148
fl_swimming_pool                

In [6]:
df.columns

Index(['id', 'price', 'property_type', 'subproperty_type', 'region',
       'province', 'locality', 'zip_code', 'latitude', 'longitude',
       'construction_year', 'total_area_sqm', 'surface_land_sqm',
       'nbr_frontages', 'nbr_bedrooms', 'equipped_kitchen', 'fl_furnished',
       'fl_open_fire', 'fl_terrace', 'terrace_sqm', 'fl_garden', 'garden_sqm',
       'fl_swimming_pool', 'fl_floodzone', 'state_building',
       'primary_energy_consumption_sqm', 'epc', 'heating_type',
       'fl_double_glazing', 'cadastral_income'],
      dtype='object')

In [7]:
# Creating lists for numerical variables, dummy variables (1/0), and categorical variables
cat_features = ['property_type', 'subproperty_type', 'region', 'province', 'locality', 'zip_code', 'state_building', 
            'epc', 'heating_type', 'equipped_kitchen']
num_features = ["cadastral_income","surface_land_sqm", "total_area_sqm", "latitude", "longitude", "garden_sqm", 
          "primary_energy_consumption_sqm", "construction_year", "nbr_frontages", "nbr_bedrooms", "terrace_sqm" ]
dummy_features = ["fl_garden", "fl_furnished", "fl_open_fire", "fl_terrace","fl_swimming_pool", "fl_floodzone", "fl_double_glazing"]


## Dealing with missing values in numeric variables 
Replacing missing values in numerical variables with the median is considered because median is less sensitive to outliers than the mean, especially when the distribution of the data is skewed.

In [8]:
# Creating the imputer for numerical data, specifying 'median' as the strategy
num_imputer = SimpleImputer(strategy='median')

# Applying the imputer to the numerical columns
df[num_features] = num_imputer.fit_transform(df[num_features])

# Check if there are any missing values left in the numerical columns
missing_values_after_imputation = df[num_features].isnull().sum()

# Displaying the number of missing values after imputation
missing_values_after_imputation


cadastral_income                  0
surface_land_sqm                  0
total_area_sqm                    0
latitude                          0
longitude                         0
garden_sqm                        0
primary_energy_consumption_sqm    0
construction_year                 0
nbr_frontages                     0
nbr_bedrooms                      0
terrace_sqm                       0
dtype: int64

## Dealing with missing values in categorical variables 
As you can see we replaced MISSING values in categorical columns with None. 

In [9]:
# Replace missing values in categorical variables with 'None'
df[cat_features].replace('MISSING' , None)

# Verify if there are any missing values left in the categorical columns
missing_values_categorical_after_replacement = df[cat_features].isnull().sum()

# Display the check result
missing_values_categorical_after_replacement


property_type       0
subproperty_type    0
region              0
province            0
locality            0
zip_code            0
state_building      0
epc                 0
heating_type        0
equipped_kitchen    0
dtype: int64

## Dealing with missing values in dummy variables 

In [10]:
# Imputing missing values on f1 features values
df[dummy_features].isnull().sum()

fl_garden            0
fl_furnished         0
fl_open_fire         0
fl_terrace           0
fl_swimming_pool     0
fl_floodzone         0
fl_double_glazing    0
dtype: int64

## Converting categorical variables to the numerical ones

In [11]:
# Initialize OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

# Fit and transform the categorical columns
#encoded_categorical = encoder.fit_transform(df[cat_features]).toarray()
encoded_categorical = encoder.fit_transform(df[cat_features]).toarray()


# Create a DataFrame for the encoded variables
encoded_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(cat_features))

# Concatenate the encoded variables back to the original DataFrame
# First, reset the index of df to align with the index of encoded_df
df.reset_index(drop=True, inplace=True)

# Concatenate
df_encoded = pd.concat([df, encoded_df], axis=1)

# Drop original categorical columns
df_encoded.drop(columns=cat_features, inplace=True)

df_encoded.head()


,id,price,latitude,longitude,construction_year,total_area_sqm,surface_land_sqm,nbr_frontages,nbr_bedrooms,fl_furnished,...,heating_type_WOOD,equipped_kitchen_HYPER_EQUIPPED,equipped_kitchen_INSTALLED,equipped_kitchen_MISSING,equipped_kitchen_NOT_INSTALLED,equipped_kitchen_SEMI_EQUIPPED,equipped_kitchen_USA_HYPER_EQUIPPED,equipped_kitchen_USA_INSTALLED,equipped_kitchen_USA_SEMI_EQUIPPED,equipped_kitchen_USA_UNINSTALLED
0,34221000,225000,51.217172,4.379982,1963.0,100.0,362.0,2.0,2.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2104000,449000,51.174944,3.845248,1994.0,127.0,680.0,3.0,2.0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34036000,335000,50.842043,4.334543,1994.0,142.0,362.0,2.0,3.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,58496000,501000,51.238312,4.817192,2024.0,187.0,505.0,3.0,3.0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,48727000,982700,50.900919,4.376713,2022.0,169.0,362.0,2.0,2.0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



## Standardizing Numeric Features with StandardScaler

In [12]:
# Initialize the StandardScaler
#scaler = StandardScaler()

# Standardize the numeric features
#df_encoded[num_features] = scaler.fit_transform(df_encoded [num_features])

# Print the standardized DataFrame
#print(df_encoded.head())


In [13]:
# Save df_encoded to a CSV file
df_encoded.to_csv('data/encoded_data.csv', index=False)

In [14]:
# Calculate the correlation of these features with the 'price' column
#corr = df[num_features + ['price']].corr()['price'].drop('price')
#corr

# Model Training 

### LinearRegression - Training model on Numerical Columns only 

In [15]:
#df = pd.read_csv("./data/encoded_data.csv")
cat_features = ['property_type', 'subproperty_type', 'region', 'province', 'locality', 'zip_code', 'state_building', 
            'epc', 'heating_type', 'equipped_kitchen']
num_features = ["cadastral_income","surface_land_sqm", "total_area_sqm", "latitude", "longitude", "garden_sqm", 
          "primary_energy_consumption_sqm", "construction_year", "nbr_frontages", "nbr_bedrooms", "terrace_sqm" ]
dummy_features = ["fl_garden", "fl_furnished", "fl_open_fire", "fl_terrace","fl_swimming_pool", "fl_floodzone","fl_double_glazing"]

In [16]:
df_encoded.columns
df_encoded.isna().sum()

id                                     0
price                                  0
latitude                               0
longitude                              0
construction_year                      0
                                      ..
equipped_kitchen_SEMI_EQUIPPED         0
equipped_kitchen_USA_HYPER_EQUIPPED    0
equipped_kitchen_USA_INSTALLED         0
equipped_kitchen_USA_SEMI_EQUIPPED     0
equipped_kitchen_USA_UNINSTALLED       0
Length: 1215, dtype: int64

In [17]:
X = df_encoded[num_features]
y = df_encoded['price']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of y_test: ", y_test.shape)

regressor = LinearRegression()
regressor.fit(X_train,y_train)
regressor.score(X_train, y_train)
print("train_score: ", regressor.score(X_train, y_train))
regressor.score(X_test, y_test)
print("test_score: ", regressor.score(X_test, y_test))

Shape of X_train:  (60408, 11)
Shape of X_test:  (15103, 11)
Shape of y_train:  (60408,)
Shape of y_test:  (15103,)
train_score:  0.1832729084881085
test_score:  0.21614822846202175


## XGboost

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Create XGBoost regression instances
xgb_regressor = xgb.XGBRegressor()

# Fit the models to the training data
xgb_regressor.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [20]:
# Predict on the test data
y_pred = xgb_regressor.predict(X_test)

# Calculate R-squared for apartment and house models
r2 = r2_score(y_test, y_pred)

# Print the R-squared scores
print("R-squared:", r2)


R-squared: 0.667055002420012


In [21]:
# Save the trained model
joblib.dump(regressor, 'trained_model_1.pkl')

['trained_model_1.pkl']